In [1]:
import re, string, csv, unicodedata, os, json, subprocess, pprint, pickle, decimal
from extract_rtf import striprtf

# Make sure 70.25 is rounded up to 70.3
decimal.setcontext(decimal.BasicContext)

pp = pprint.PrettyPrinter(indent=4).pprint

PARTS = ['tenor', 'lead', 'bari', 'bass']
CATS = {
    'Music': 'm',
    'Performance': 'p',
    'Singing': 's',
    'CA': 'a'
}

In [2]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

In [3]:
# Iterate through files in a list of directories
def getfiles(directories, ext):
    for directory in directories:
        for filename in os.listdir(directory):
            if filename.endswith(ext):
                yield os.path.join(directory, filename)

# test
#list(getfiles(['BABS RTF'],'rtf'))

In [4]:
# Change a file extension
def change_ext(filename, ext):
    return('%s.%s' % (os.path.splitext(filename)[0], ext))
    
# test
#print(change_ext('BABS RTF\\Results-1996-97-QF-NAT-Detail.anyarbitraryextension', 'rtf'))

In [5]:
# Convert all files in a directory from rtf to plain text
def convert_all_rft2txt(directory):
    for infile in getfiles([directory], 'rtf'):
        outfile = change_ext(infile, 'txt')
        try:
            with open(infile, 'rb') as f:
                plain_text = striprtf(f.read())
            with open(outfile, 'w') as f:
                f.write(plain_text)
        except Exception as e:
            print(infile)
            print(e)
            pass

# test        
convert_all_rft2txt('BABS RTF')        

In [6]:
def parse_contestant(text):
    """
    Parse text for one contestant, in this format:
    1: Cambridge Blues	Dear Old Girl	235	211	218
    	I Got Rhythm	211	206	210	1291	71.7
    	Previous (balanced):	440.0	440.0	411.0	2582.0
    """

    # Assemble the string containing rank, name, and members (usually split across several lines)
    # (Get all the text before the first tab on each line, and concatenate it)
    r = re.compile(r'^[^\t]+', re.MULTILINE)
    contestant = parse_rank_name_members("".join(m2.group(0) for m2 in r.finditer(text)))

    # Get the strings containing the scores
    # (Get all the text after the first tab on each line)
    r = re.compile(r'\t(.+)$', re.MULTILINE)
    contestant['songs'] = [parse_song(m.group(1)) for m in r.finditer(text)]
                
    return contestant    

In [7]:
def parse_num(text):
    try:
        return int(text.strip())
    except ValueError:
        return decimal.Decimal(text.strip())

In [8]:
def parse_song(text):
    """
    Extract song title and scores from a string in one of these formats:
    Dear Old Girl	235	211	218
    I Got Rhythm	211	206	210	1291	71.7
    Previous (balanced):	440.0	440.0	411.0	2582.0
    My Wife The Dancer	175	167	(- 13)	164	1015	56.4
    """
    penalty = re.compile(r'\(- (\d+)\)')
    song = {'n': 1}
    
    # Split to a list of strings, then pop items off the list as it is parsed
    split = re.split('\t', text)    
    
    # If the first item in the list is a number, it means there is no title
    song['name'] = None if split[0].isdigit() else split.pop(0)
    if not song['name']:    
        print("warning - no song title")
    
    # Get the music score/penalty
    song['m'] = parse_num(split.pop(0))
    m = penalty.match(split[0])
    if m:
        song['mr'] = -parse_num(m.group(1))
        split.pop(0)
    
    # Get performance score/penalty
    song['p'] = parse_num(split.pop(0))
    m = penalty.match(split[0])
    if m:
        song['pr'] = -parse_num(m.group(1))
        split.pop(0)
    
    # Get singing score
    song['s'] = parse_num(split.pop(0))
                
    return song    

In [9]:
def parse_rank_name_members(text):
    """
    Extracts rank, contestant name, and members, from a string in one of these formats:
    1: Cambridge Blues
    1: Hallmark Of Harmony (81)
    1: The Great Western Chorus Of Bristol  (Linda Corcoran) (52)
    1: RECKLESS  (Andy Foster, Duncan Whinyates, Dale Kynaston, Andy Funnell)
    Returns a contestant object
    """    
    # Extract rank and name (same for all types of contest)
    # Assumes contestant name doesn't contain any brackets
    m = re.match(r'(\d+): ([^\(]+)', text)
    contestant = {
        'rank': int(m.group(1)),
        'name': m.group(2).strip()
    }

    # Extract any text within brackets
    for m in re.finditer(r'\((.+?)\)', text):
        text_in_brackets = m.group(1).strip()
        try:
            # Check if it's the chorus size (an integer)
            contestant['size'] = int(text_in_brackets)
        except ValueError:
            # Split the string into seprate names.
            names = re.split(r'\s*(?: and |&|,|;|/)\s*', text_in_brackets)
            # 4 names = quartet members, less than 4 names = chorus director(s)
            if len(names) == 4:
                contestant['members'] = [{'part': PARTS[i], 'name': names[i]} for i in range(4)]
            else:
                contestant['members'] = [{'part': 'director', 'name': name} for name in names]

    return contestant

In [10]:
def calculate_scores(contest):
    """
    Fill in the missing scores for a contest object, e.g. category total scores and song percentages
    :param contest: A contest object
    :return: The modified contest object
    """
    CATS = ('m', 'p', 's')

    # Very quick and dirty hack to deal with rolling panels - assuming only half the judges are on the panel at a time
    # Note str.find returns -1 if substring is not found
    # rolling_panel_factor = 1 if contest['raw_text'].find('Rolling Panel:') == -1 else 2

    # Calculate number of judges excluding administrators
    n_judges_by_cat = {cat: sum(1 for j in contest['judges'] if j['cat'] == cat) for cat in CATS}
    assert(n_judges_by_cat['m'] == n_judges_by_cat['p'] == n_judges_by_cat['s'])
    n_judges = sum(n_judges_by_cat.values())
    n_judges_per_cat = n_judges_by_cat['m']

    # Loop through contestants
    for contestant in contest['contestants']:

        # calculate the total score, total percentage, and category percentages for each song
        for song in contestant['songs']:

            # calculate song total score = sum of category scores
            song['tot_score'] = sum(song[cat] for cat in CATS)

            # Delete any songs with zero score
            if song['tot_score'] == 0:
                contestant['songs'].remove(song)
                print('deleted zero score')
                continue
                
            # Workaround to make sure that "Previous" scores in the BABS/LABBS scoresheets
            # are counted as two songs when calculating percentages
            song['n'] = 1 if (song['name'] is None or "Previous" not in song['name']) else 2

            # calculate song percentage score = total score / number of judges
            song['pc_score'] = round(decimal.Decimal(song['tot_score']) / song['n'] / n_judges, 1)

            # calculate song category percentage scores = category score / number of judges per cat
            for cat in CATS:
                song['%s_pc' % cat] = round(decimal.Decimal(song[cat]) / song['n'] / n_judges_per_cat, 1)                

        # count the number of songs that the contestant sang (used for calculating contestant percentages
        contestant['n'] = sum(song['n'] for song in contestant['songs'])

        # calculate the category totals and percentages for each contestant
        for cat in CATS:
            contestant[cat] = sum(song[cat] for song in contestant['songs'])
            contestant['%s_pc' % cat] = round(decimal.Decimal(contestant[cat]) / contestant['n'] / n_judges_per_cat, 1)
    
        # Calculate total score and pc score
        try:
            contestant['tot_score'] = sum(song['tot_score'] for song in contestant['songs'])
        except Exception as e:
            print(contestant['songs'])
            raise e
        contestant['pc_score'] = round(decimal.Decimal(contestant['tot_score']) / contestant['n'] / n_judges, 1)
            


In [11]:
#Contestant	Songs	Mus	MR	Perf	PR	Sing	Ch.Pen	Totals	%
#1: Hallmark Of Harmony (81)	Let The End Of The World Come Tomorrow	233	231	235
#	I'm Looking Over A Four Leaf Clover	235	246	236	1416	78.7
#	Category rankings:	1	1	1

#Contestant	Songs	Mus	MR	Perf	PR	Sing	Ch.Pen	Totals	%
#1: Cambridge Blues	Dear Old Girl	235	211	218
#	I Got Rhythm	211	206	210	1291	71.7
#	Previous (balanced):	440.0	440.0	411.0	2582.0
#	Category rankings:	1	1	1

contests = []

for filename in getfiles(['BABS RTF'],'txt'):
    contest = {
        'assoc': 'BABS',
        'filename': filename
    }
    print('parsing %s' % filename)

    with open(filename, 'r') as f:
        plain_text = f.read()
    
    # Get the third line of text
    text = plain_text.splitlines()[2]
    r = re.compile(r'(?P<contest>.*)  -  (?P<location>.*): (?P<year>[\d/]*)')
    m = r.match(text)
    for key in ('contest', 'location', 'year'):
        contest[key] = m.group(key) if len(m.group(key)) > 0 else None
        if not contest[key]:
            print('warning - no %s' % key)
    
    # Chorus or quartet contest?
    if 'CHORUS' in contest['contest']:
        contest['type'] = 'c' 
    elif 'QUARTET' in contest['contest']:
        contest['type'] = 'q' 
    else:
        print('warning - unknown contest type')
    
    # Parse the contest date
    m = re.search('Contest date: (\d{2}/\d{2}/\d{4})', plain_text)
    contest['date'] = m.group(1) if m else None
    if not contest['date']:
        print('warning - no contest date')
        
    # Parse the judges
    contest['judges'] = []
    r = re.compile('(Music|Performance|Singing|CA): (.+)')
    for m in r.finditer(plain_text):
        # Convert 'Performance' to 'p' etc
        cat = CATS[m.group(1)]
        # Split the comma separated list of judges' names
        names = re.split(', *', m.group(2))
        # Add to list of judges
        for name in names:
            contest['judges'].append({'cat':cat, 'name': name.strip()})
            
    # Parse the contestants
    r = re.compile(r'(\d+:.*?)\n\tCategory', re.DOTALL)
    contest['contestants'] = [parse_contestant(m.group(1)) for m in r.finditer(plain_text)]
    
    # Calculate totals and percentages
    calculate_scores(contest)
    
    # Save as pickle
    with open(change_ext(filename, 'pickle'), 'wb') as outfile:
        pickle.dump(contest, outfile)
    
    # Save as json
    with open(change_ext(filename, 'json'), 'w') as outfile:
        json.dump(contest, outfile, indent=2, cls=DecimalEncoder)      
    
    # Append to the list of contests
    contests.append(contest)
    

parsing BABS RTF\Results-1996-97-CF-NAT-Detail.txt
parsing BABS RTF\Results-1996-97-CP-MID-Detail.txt
parsing BABS RTF\Results-1996-97-CP-NTH-Detail.txt
parsing BABS RTF\Results-1996-97-CP-STH-Detail.txt
parsing BABS RTF\Results-1996-97-QF-NAT-Detail.txt
parsing BABS RTF\Results-1996-97-QP-MID-Detail.txt
parsing BABS RTF\Results-1996-97-QP-NTH-Detail.txt
parsing BABS RTF\Results-1996-97-QP-STH-Detail.txt
parsing BABS RTF\Results-1996-97-QS-NAT-Detail.txt
parsing BABS RTF\Results-1997-98-CF-NAT-Detail.txt
parsing BABS RTF\Results-1997-98-CP-MID-Detail.txt
parsing BABS RTF\Results-1997-98-CP-NTH-Detail.txt
parsing BABS RTF\Results-1997-98-CP-STH-Detail.txt
parsing BABS RTF\Results-1997-98-QF-NAT-Detail.txt
parsing BABS RTF\Results-1997-98-QP-MID-Detail.txt
parsing BABS RTF\Results-1997-98-QP-NTH-Detail.txt
parsing BABS RTF\Results-1997-98-QP-STH-Detail.txt
parsing BABS RTF\Results-1997-98-QPM-SEN-Detail.txt
parsing BABS RTF\Results-1997-98-QPN-SEN-Detail.txt
parsing BABS RTF\Results-1997

KeyError: 'tot_score'

In [ ]:
# export to excel
from pandas.io.json import json_normalize
import pandas as pd

dfs = []
for contest in contests:
    dfs.append(json_normalize(contest, 'contestants', ['assoc', 'contest', 'year', 'location', 'type', 'date', 'filename']))
pd.concat(dfs).to_excel('babs by contestants.xlsx')

dfs = []
for contest in contests:
    dfs.append(json_normalize(contest, 
                   ['contestants', 'songs'], 
                   ['assoc', 'contest', 'year', 'location', 'type', 'date', 'filename',
                    ['contestant', 'name'],
                    ['contestant', 'tot_score'],
                    ['contestant', 'pc_score'],
                    ['contestant', 'rank'],
                   ]))
pd.concat(dfs).to_excel('babs by songs.xlsx')